## Load Packages

In [1]:
#using Dates       #Julia 0.7
include("printmat.jl")

printmatDate

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Portfolio Choice

Suppose you can invest in a risky asset (with expected return $\mu$ and standard deviation $\sigma$) and into a riskfree asset (at the rate $R_f$). 

With the portfolio weight $v$ on the risky asset, the average and variance of the portfolio is

$\textrm{E}R_p = v \mu + (1-v)R_f$ 

and 

$\textrm{Std}(R_p) = |v|\sigma$

In [3]:
μ  = 9.5/100  #expected return on "market"
σ  = 8/100    #std of market 
Rf = 3/100    #risk free return (interest rate)
v  = [0;0.5;1;2]  #trying different weights on risky asset 
ERp   = v*μ + (1 .- v)*Rf      #1-v is the weight on the riskfree
StdRp = abs.(v)*σ 

println()

In [4]:
scatter(StdRp*100,ERp*100,legend=false,ylim=(0,20),xlim=(0,20))
title!("Mean vs standard deviation for portfolios of a risky and a riskfree asset")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 Mean vs standard deviation for portfolios of a risky and a riskfree asset 
 
 
 Std(Rp), % 
 
 
 ERp, %

Maximizing $\textrm{E}R_p - k/2\times \textrm{Var}(R_p)$  gives the optimal portfolio weight

$v = \frac{\mu - R_f}{k\sigma^2}$

In [5]:
k = 25

v = linspace(-0.5,2,101)         #trying different portfolio weights

ERp   = v*μ + (1 .- v)*Rf
VarRp = v.^2*σ^2 
Util  = ERp - k/2*VarRp

vopt = (μ-Rf)/(k*σ^2)
printlnPs("Weight on risk and riskfree when k = $k: ",vopt," ",1-vopt)

Weight on risk and riskfree when k = 25:      0.406      0.594


In [6]:
plot(v,Util,color=:red,linewidth=2,legend=false)
title!("Utility")
xlabel!("v (weight on risky asset)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 -0.15 
 
 
 -0.10 
 
 
 -0.05 
 
 
 0.00 
 
 
 Utility 
 
 
 v (weight on risky asset) 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 45.7332,137.731 51.079,131.951 56.4249,126.332 61.7707,120.874 67.1165,115.579 72.4623,110.445 77.8081,105.473 83.1539,100.662 88.4998,96.0134 93.8456,91.5263 
 99.1914,87.2009 104.537,83.0371 109.883,79.0351 115.229,75.1948 120.575,71.5162 125.92,67.9993 131.266,64.6441 136.612,61.4505 141.958,58.4187 147.304,55.5486 
 152.65,52.8401 157.995,50.2934 163.341,47.9084 168.687,45.685 174.033,43.6234 179.379,41.7234 184.724,39.9852 190.07,38.4086 195.416,36.9938 200.762,35.7406 
 206.108,34.6492 211.454,33.7194 216.799,32.9513 222.145,32.345 227.491,31.9003 232.837,31.6173 238.183,31.4961 243.528,31.5365 248.874,31.7386 254.22,32.1024 
 259.566,32.6279 264.912,33.3152 270.258,34.1641 275.603,35.1747 280.949,36.347 286.295,37.681 291.641,39.1767 296.987,40.8341 302.332,42.6532 307.678,44.634 
 313.024,46.7765 318.37,49.0807 323.716,51.5466 329.062,54.1741 334.407,56.9634 339.753,59.9144 345.099,63.0271 350.445,66.3015 355.791,69.7375 361.136,73.3353 
 366.482,77.0948 371.828,81.0159 377.174,85.0988 382.52,89.3433 387.866,93.7496 393.211,98.3176 398.557,103.047 403.903,107.939 409.249,112.992 414.595,118.206 
 419.94,123.583 425.286,129.121 430.632,134.821 435.978,140.682 441.324,146.706 446.67,152.89 452.015,159.237 457.361,165.745 462.707,172.415 468.053,179.247 
 473.399,186.241 478.744,193.396 484.09,200.713 489.436,208.191 494.782,215.831 500.128,223.633 505.474,231.597 510.819,239.722 516.165,248.009 521.511,256.458 
 526.857,265.068 532.203,273.84 537.548,282.774 542.894,291.869 548.24,301.127 553.586,310.546 558.932,320.126 564.278,329.868 569.623,339.772 574.969,349.838 
 580.315,360.065 
 "/>

#    Diversification

The variance of an equally weighted portfolio is

$\sigma^2_p = (\sigma_{ii} - \sigma_{ij})/n + \sigma_{ij}$,

where $\sigma_{ii}$ is the average variance (across the assets) and $\sigma_{ij}$ is the average covariance across the assets.

In [7]:
σii = 0.3      #average variance
σij = 0.05     #average covariance
n = 1:49
VarRp = (σii-σij)./n .+ σij     #variance of equally weighted portfolio

println()

In [8]:
plot(n,VarRp,color=:red,linewidth=2,legend=false,ylim=(0,0.3))
title!("Variance of equally weighted portfolio")
xlabel!("n (number of assets)")
ylabel!("Variance")
plot!([σij;σij],linetype=:hline,line=(:dash,1),color=:blue)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 Variance of equally weighted portfolio 
 
 
 n (number of assets) 
 
 
 Variance 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 54.5598,31.4961 65.513,168.4 76.4662,214.035 87.4195,236.852 98.3727,250.542 109.326,259.669 120.279,266.188 131.232,271.078 142.186,274.881 153.139,277.923 
 164.092,280.412 175.045,282.487 185.999,284.242 196.952,285.746 207.905,287.05 218.858,288.191 229.811,289.198 240.765,290.092 251.718,290.893 262.671,291.613 
 273.624,292.265 284.578,292.858 295.531,293.399 306.484,293.895 317.437,294.352 328.391,294.773 339.344,295.163 350.297,295.525 361.25,295.862 372.204,296.177 
 383.157,296.471 394.11,296.747 405.063,297.007 416.016,297.251 426.97,297.481 437.923,297.698 448.876,297.904 459.829,298.098 470.783,298.283 481.736,298.459 
 492.689,298.626 503.642,298.785 514.596,298.936 525.549,299.081 536.502,299.219 547.455,299.352 558.408,299.478 569.362,299.6 580.315,299.716 
 "/>